# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_info = pd.read_csv("output_data/cities.csv")
city_info.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,namibe,-15.1961,12.1522,75.29,44,0,10.22,AO,1623026179
1,1,chokurdakh,70.6333,147.9167,41.27,60,76,5.68,RU,1623026179
2,2,mataura,-46.1927,168.8643,49.48,65,0,3.78,NZ,1623026180
3,3,cape town,-33.9258,18.4232,68.00,34,0,16.11,ZA,1623025899
4,4,katsuura,35.1333,140.3000,73.42,85,94,2.64,JP,1623026180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of humidity
locations = city_info[["Lat", "Lng"]]
humidity = city_info["Humidity"]
fig = gmaps.figure()
heat_point = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_point)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_cities = city_info.loc[(city_info["Max Temp"] < 80) & (city_info["Max Temp"] > 70) \
                                    & (city_info["Wind Speed"] < 10) \
                                    & (city_info["Cloudiness"] == 0)].dropna()
narrowed_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
196,196,saquarema,-22.9200,-42.5103,75.20,78,0,3.44,BR,1623026269
259,259,pishin,30.5803,66.9961,75.04,51,0,3.40,PK,1623026297
402,402,hambantota,6.1241,81.1185,75.20,100,0,3.44,LK,1623026364
450,450,vila velha,-20.3297,-40.2925,73.00,88,0,2.30,BR,1623026386
464,464,maceio,-9.6658,-35.7353,72.84,94,0,4.25,BR,1623026379
519,519,santarem,-2.4431,-54.7083,78.93,94,0,4.61,BR,1623026199


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = narrowed_cities[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
24,nikolskoye,RU,59.7035,30.7861,
26,port alfred,ZA,-33.5906,26.8910,
48,birecik,TR,37.0294,37.9903,
86,port elizabeth,ZA,-33.9180,25.5701,
99,belyy yar,RU,53.6039,91.3903,
183,gat,IL,31.6100,34.7642,
196,saquarema,BR,-22.9200,-42.5103,
210,changji,CN,44.0167,87.3167,
216,linjiang,CN,41.8069,126.9078,
230,marsa matruh,EG,31.3525,27.2453,


In [10]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

,City,Country,Lat,Lng,Hotel Name
24,nikolskoye,RU,59.7035,30.7861,
26,port alfred,ZA,-33.5906,26.8910,
48,birecik,TR,37.0294,37.9903,
86,port elizabeth,ZA,-33.9180,25.5701,
99,belyy yar,RU,53.6039,91.3903,
183,gat,IL,31.6100,34.7642,
196,saquarema,BR,-22.9200,-42.5103,
210,changji,CN,44.0167,87.3167,
216,linjiang,CN,41.8069,126.9078,
230,marsa matruh,EG,31.3525,27.2453,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))